In [1]:
#import nltk
#nltk.download('stopwords')

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
#cufflinks.go_offline()
#cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [3]:
df1 = pd.read_csv('data_sgd/train.tsv', header=None, sep="\t")

In [4]:
df1.head()

,0,1
0,__label__1,"sounds good, i want to buy tickets."
1,__label__3,please search a hotel for me there
2,__label__4,nothing. just relax.
3,__label__5,shall i book a table here?
4,__label__6,"no, not now.i need to check my account balance?"


In [5]:
df2 = pd.read_csv('data_sgd/test.tsv', header=None, sep="\t")
df2.head()

,0,1
0,__label__23,sounds good. can you find me a rental car ther...
1,__label__26,i need to check weather on the 5th of march.
2,__label__24,cool. schedule a visit.
3,__label__14,i would like to get a cab going there.
4,__label__1,would you like some tickets?


In [6]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 80#250
# This is fixed.
EMBEDDING_DIM = 100
HIDDEN_DIM = 64

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1[1].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 4476 unique tokens.


In [7]:
X1 = tokenizer.texts_to_sequences(df1[1].values)
X1 = pad_sequences(X1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X1.shape)

Shape of data tensor: (47022, 80)


In [8]:
Y1 = pd.get_dummies(df1[0]).values
print('Shape of label tensor:', Y1.shape)

Shape of label tensor: (47022, 21)


In [9]:
X2 = tokenizer.texts_to_sequences(df2[1].values)
X2 = pad_sequences(X2, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X2.shape)

Shape of data tensor: (9838, 80)


In [10]:
Y2 = pd.get_dummies(df2[0]).values
print('Shape of label tensor:', Y2.shape)

Shape of label tensor: (9838, 21)


In [11]:
X_train, X_test, Y_train, Y_test = X1,X2, Y1,Y2
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(47022, 80) (47022, 21)
(9838, 80) (9838, 21)


In [12]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [13]:
from keras.layers import GRU, RNN, Bidirectional

In [14]:
#model.add(Bidirectional(LSTM(64)))

#model = Sequential()
#model.add(Embedding(max_num_words, 100, input_length=max_length))
#model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))

In [15]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X1.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(HIDDEN_DIM, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m]) # metrics=['accuracy']
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 100)           5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 80, 100)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_1 (Dense)              (None, 21)                2709      
Total params: 5,087,189
Trainable params: 5,087,189
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 42319 samples, validate on 4703 samples
Epoch 1/5





42319/42319 [==============================] - 352s 8ms/step - loss: 1.1015 - acc: 0.6974 - f1_m: 0.6459 - precision_m: 0.8644 - recall_m: 0.5625 - val_loss: 0.3971 - val_acc: 0.8811 - val_f1_m: 0.8772 - val_precision_m: 0.9142 - val_recall_m: 0.8437
Epoch 2/5
42319/42319 [==============================] - 347s 8ms/step - loss: 0.3431 - acc: 0.8881 - f1_m: 0.8877 - precision_m: 0.9167 - recall_m: 0.8608 - val_loss: 0.3346 - val_acc: 0.8848 - val_f1_m: 0.8866 - val_precision_m: 0.9098 - val_recall_m: 0.8650
Epoch 3/5
42319/42319 [==============================] - 347s 8ms/step - loss: 0.2884 - acc: 0.8979 - f1_m: 0.8993 - precision_m: 0.9220 - recall_m: 0.8780 - val_loss: 0.3118 - val_acc: 0.8924 - val_f1_m: 0.8943 - val_precision_m: 0.9144 - val_recall_m: 0.8754
Epoch 4/5
42319/42319 [==============================] - 347s 8ms/

In [17]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

9838/9838 [==============================] - 9s 904us/step
Test set
  Loss: 0.290
  Accuracy: 0.901


In [18]:
#loss, accuracy, f1_score, precision, recall
print(accr)

[0.2900172078856658, 0.9014027240945692, 0.9071156883850454, 0.9248031864783753, 0.8906281764222781]


In [19]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(np.argmax(Y_test, axis=1), y_pred_bool, digits=4))

9838/9838 [==============================] - 10s 978us/step
              precision    recall  f1-score   support

           0     0.8443    0.8197    0.8318       549
           1     0.9125    0.9812    0.9456       319
           2     0.9492    0.9333    0.9412       180
           3     0.9144    0.9889    0.9502       270
           4     0.9500    0.9694    0.9596       196
           5     0.9035    0.8420    0.8717       367
           6     0.8706    0.5382    0.6652       275
           7     0.9511    0.8333    0.8883       210
           8     0.9454    0.9590    0.9522       415
           9     0.8909    0.9304    0.9102       158
          10     0.7108    0.9712    0.8209       243
          11     0.9072    0.9580    0.9319       357
          12     0.7605    0.8485    0.8021       363
          13     0.8087    0.9376    0.8684       577
          14     0.8719    0.7099    0.7826       393
          15     0.9811    0.9765    0.9788       213
          16     0.96